# Prediction of R2L attacks

In [1]:
import pandas as p

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# feature names
features = ["duration", "protocol_type", "service", "flag", "src_bytes", "dst_bytes", "land", "Wrong_fragment", "Urgent", "hot", "num_failed_login", "logged_in", "num_compromised", "root_shell", "su_attempted", "num_root", "num_file_creations", "num_shells", "num_access_files", "num_outbound_cmds", "is_host_login", "is_guest_login", "count", "srv_count", "serror_rate", "srv_serror_rate", "rerror_rate", "srv_rerror_rate", "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate", "dst_host_count", "dst_host_srv_count", "dst_host_same_srv_rate", "dst_host_diff_ srv_rate", "dst_host_same_src_port_rate", "dst_host_srv_diff_host _rate", "dst_host_serror_rate", "dst_host_srv_serror_rate", "dst_host_rerror_rate", "dst_host_srv_rerror_rate","class"] 
data = p.read_csv("data6.csv", names = features)

In [4]:
df=data.dropna()

In [5]:
df['R2L'] = df["class"].map({'normal.':0, 'snmpgetattack.':1, 'named.':1, 'xlock.':1, 'smurf.':0,
       'ipsweep.':0, 'multihop.':1, 'xsnoop.':1, 'sendmail.':1, 'guess_passwd.':1,
       'saint.':0, 'buffer_overflow.':0, 'portsweep.':0, 'pod.':0, 'apache2.':0,
       'phf.':1, 'udpstorm.':0, 'warezmaster.':1, 'perl.':0, 'satan.':0, 'xterm.':0,
       'mscan.':0, 'processtable.':0, 'ps.':0, 'nmap.':0, 'rootkit.':0, 'neptune.':0,
       'loadmodule.':0, 'imap.':1, 'back.':0, 'httptunnel.':1, 'worm.':1,
       'mailbomb.':0, 'ftp_write.':1, 'teardrop.':0, 'land.':0, 'sqlattack.':0,
       'snmpguess.':1})

In [6]:
from sklearn.preprocessing import LabelEncoder
var_mod = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'Wrong_fragment', 'Urgent', 'hot',
       'num_failed_login', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_ srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host _rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate'
       ]
le = LabelEncoder()
for i in var_mod:
    df[i] = le.fit_transform(df[i]).astype(int)


In [7]:
del df['dst_host_srv_rerror_rate']
del df["class"]

In [8]:
#According to the cross-validated MCC scores, the random forest is the best-performing model, so now let's evaluate its performance on the test set.
from sklearn.metrics import confusion_matrix, classification_report, matthews_corrcoef, cohen_kappa_score, accuracy_score, average_precision_score, roc_auc_score

In [9]:
X = df.drop(labels='R2L', axis=1)
#Response variable
y = df.loc[:,'R2L']    

In [10]:
 #We'll use a test size of 30%. We also stratify the split on the response variable, which is very important to do because there are so few fraudulent transactions.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24, stratify=y)

In [11]:
#According to the cross-validated MCC scores, the random forest is the best-performing model, so now let's evaluate its performance on the test set.
from sklearn.metrics import confusion_matrix, classification_report, matthews_corrcoef, cohen_kappa_score, accuracy_score, average_precision_score, roc_auc_score

Logistic Regression :

In [12]:

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

logR= LogisticRegression()
logR.fit(X_train,y_train)
predictR = logR.predict(X_test)
print("")
print('Classification report of Logistic Regression Results:')
print("")
print(classification_report(y_test,predictR))

cm1=confusion_matrix(y_test,predictR)
print('Confusion Matrix result of Logistic Regression is:\n',cm1)
print("")
sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
print('Sensitivity : ', sensitivity1 )
print("")
specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
print('Specificity : ', specificity1)
print("")

accuracy = cross_val_score(logR, X, y, cv=10, scoring='accuracy')
print('Cross validation test results of accuracy:')
print(accuracy)
#get the mean of each fold 
print("")
print("Accuracy result of Logistic Regression is:",accuracy.mean() * 100)
lr=accuracy.mean() * 100


Classification report of Logistic Regression Results:

              precision    recall  f1-score   support

           0       0.62      0.94      0.75       357
           1       0.63      0.16      0.26       243

    accuracy                           0.62       600
   macro avg       0.63      0.55      0.50       600
weighted avg       0.63      0.62      0.55       600

Confusion Matrix result of Logistic Regression is:
 [[334  23]
 [203  40]]

Sensitivity :  0.9355742296918768

Specificity :  0.1646090534979424

Cross validation test results of accuracy:
[0.61129568 0.62458472 0.63787375 0.63666667 0.62333333 0.66333333
 0.61333333 0.64882943 0.60535117 0.62541806]

Accuracy result of Logistic Regression is: 62.90019481697946


Decision Tree:

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score

dt= DecisionTreeClassifier()
dt.fit(X_train,y_train)
predictR = dt.predict(X_test)
print("")
print('Classification report of Decision Tree Results:')
print("")
print(classification_report(y_test,predictR))

cm1=confusion_matrix(y_test,predictR)
print('Confusion Matrix result of Decision Tree is:\n',cm1)
print("")
sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
print('Sensitivity : ', sensitivity1 )
print("")
specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
print('Specificity : ', specificity1)
print("")

accuracy = cross_val_score(dt, X, y, cv=10, scoring='accuracy')
print('Cross validation test results of accuracy:')
print(accuracy)
#get the mean of each fold 
print("")
print("Accuracy result of Decision Tree is:",accuracy.mean() * 100)
dt=accuracy.mean() * 100


Classification report of Decision Tree Results:

              precision    recall  f1-score   support

           0       0.62      0.90      0.74       357
           1       0.58      0.20      0.30       243

    accuracy                           0.62       600
   macro avg       0.60      0.55      0.52       600
weighted avg       0.61      0.62      0.56       600

Confusion Matrix result of Decision Tree is:
 [[322  35]
 [194  49]]

Sensitivity :  0.9019607843137255

Specificity :  0.20164609053497942

Cross validation test results of accuracy:
[0.64451827 0.63122924 0.63122924 0.60333333 0.64       0.64333333
 0.59333333 0.61538462 0.61538462 0.64548495]

Accuracy result of Decision Tree is: 62.63230924788053


Random Forest:

In [14]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
predictR = rfc.predict(X_test)
print("")
print('Classification report of Random Forest Results:')
print("")
print(classification_report(y_test,predictR))
print("")
cm1=confusion_matrix(y_test,predictR)
print('Confusion Matrix result of Random Forest is:\n',cm1)
print("")
sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
print('Sensitivity : ', sensitivity1 )
print("")
specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
print('Specificity : ', specificity1)
print("")

accuracy = cross_val_score(rfc, X, y, cv=10, scoring='accuracy')
print('Cross validation test results of accuracy:')
print(accuracy)
#get the mean of each fold 
print("")
print("Accuracy result of Random Forest is:",accuracy.mean() * 100)
rf=accuracy.mean() * 100


Classification report of Random Forest Results:

              precision    recall  f1-score   support

           0       0.63      0.89      0.74       357
           1       0.59      0.23      0.33       243

    accuracy                           0.62       600
   macro avg       0.61      0.56      0.53       600
weighted avg       0.61      0.62      0.57       600


Confusion Matrix result of Random Forest is:
 [[318  39]
 [188  55]]

Sensitivity :  0.8907563025210085

Specificity :  0.22633744855967078

Cross validation test results of accuracy:
[0.64119601 0.62790698 0.62790698 0.60333333 0.63333333 0.65333333
 0.60333333 0.61204013 0.62876254 0.63545151]

Accuracy result of Random Forest is: 62.6659748071275


Support Vector Classifier:

In [15]:
from sklearn.svm import SVC
sv = SVC()
sv.fit(X_train, y_train)
predictSVC = sv.predict(X_test)
print("")
print('Classification report of Support Vector Classifier Results:')
print("")
print(classification_report(y_test,predictSVC))

print("")
cm4=confusion_matrix(y_test,predictSVC)
print('Confusion Matrix result of Support Vector Classifier is:\n', confusion_matrix(y_test,predictSVC))
print("")
sensitivity1 = cm4[0,0]/(cm4[0,0]+cm4[0,1])
print('Sensitivity : ', sensitivity1 )
print("")
specificity1 = cm4[1,1]/(cm4[1,0]+cm4[1,1])
print('Specificity : ', specificity1)

accuracy = cross_val_score(sv, X, y, cv=10, scoring='accuracy')
print('Cross validation test results of accuracy:')
print(accuracy)
#get the mean of each fold 
print("")
print("Accuracy result of Support Vector Classifier is:",accuracy.mean() * 100)
sv=accuracy.mean() * 100


Classification report of Support Vector Classifier Results:

              precision    recall  f1-score   support

           0       0.62      0.93      0.74       357
           1       0.59      0.16      0.25       243

    accuracy                           0.61       600
   macro avg       0.61      0.54      0.49       600
weighted avg       0.61      0.61      0.54       600


Confusion Matrix result of Support Vector Classifier is:
 [[331  26]
 [205  38]]

Sensitivity :  0.927170868347339

Specificity :  0.15637860082304528
Cross validation test results of accuracy:
[0.60797342 0.62790698 0.64451827 0.63333333 0.63333333 0.66333333
 0.60333333 0.64214047 0.59531773 0.62541806]

Accuracy result of Support Vector Classifier is: 62.76608258610281


Naive Bayes:

In [16]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train,y_train)
predictR = gnb.predict(X_test)
print("")
print('Classification report of Naive Bayes Results:')
print("")
print(classification_report(y_test,predictR))

print("")
cm1=confusion_matrix(y_test,predictR)
print('Confusion Matrix result of Naive Bayes is:\n',cm1)
print("")
sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
print('Sensitivity : ', sensitivity1 )
print("")
specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
print('Specificity : ', specificity1)
print("")

accuracy = cross_val_score(gnb, X, y, cv=10, scoring='accuracy')
print('Cross validation test results of accuracy:')
print(accuracy)
#get the mean of each fold 
print("")
print("Accuracy result of  Naive Bayes is:",accuracy.mean() * 100)
nb=accuracy.mean() * 100


Classification report of Naive Bayes Results:

              precision    recall  f1-score   support

           0       0.75      0.03      0.05       357
           1       0.41      0.99      0.58       243

    accuracy                           0.41       600
   macro avg       0.58      0.51      0.31       600
weighted avg       0.61      0.41      0.26       600


Confusion Matrix result of Naive Bayes is:
 [[  9 348]
 [  3 240]]

Sensitivity :  0.025210084033613446

Specificity :  0.9876543209876543

Cross validation test results of accuracy:
[0.41528239 0.42524917 0.41860465 0.42666667 0.41666667 0.42333333
 0.43666667 0.41471572 0.43143813 0.43143813]

Accuracy result of  Naive Bayes is: 42.40061519202065


K-Nearest Neighbor:

In [17]:
from sklearn.neighbors import KNeighborsClassifier
knnc = KNeighborsClassifier()
knnc.fit(X_train,y_train)
predictR = knnc.predict(X_test)
print("")
print('Classification report of K-Nearest Neighbor Results:')
print("")
print(classification_report(y_test,predictR))

print("")
cm2=confusion_matrix(y_test,predictR)
print('Confusion Matrix result of K-Nearest Neighbor is:\n',cm2)
print("")
sensitivity1 = cm2[0,0]/(cm2[0,0]+cm2[0,1])
print('Sensitivity : ', sensitivity1 )
print("")
specificity1 = cm2[1,1]/(cm2[1,0]+cm2[1,1])
print('Specificity : ', specificity1)
print("")

accuracy = cross_val_score(knnc, X, y, cv=10, scoring='accuracy')
print('Cross validation test results of accuracy:')
print(accuracy)
#get the mean of each fold 
print("")
print("Accuracy result of  K-Nearest Neighbor is:",accuracy.mean() * 100)
kn=accuracy.mean() * 100


Classification report of K-Nearest Neighbor Results:

              precision    recall  f1-score   support

           0       0.67      0.73      0.69       357
           1       0.54      0.47      0.50       243

    accuracy                           0.62       600
   macro avg       0.60      0.60      0.60       600
weighted avg       0.61      0.62      0.61       600


Confusion Matrix result of K-Nearest Neighbor is:
 [[259  98]
 [130 113]]

Sensitivity :  0.7254901960784313

Specificity :  0.46502057613168724

Cross validation test results of accuracy:
[0.62790698 0.61461794 0.41196013 0.42       0.62333333 0.45666667
 0.58333333 0.57859532 0.56521739 0.66220736]

Accuracy result of  K-Nearest Neighbor is: 55.438384500568525


In [18]:
def graph():
    import matplotlib.pyplot as plt
    data=[lr,dt,rf,sv,nb,kn]
    alg="LR","DT","RF","SVM","NB","KNN"
    plt.figure(figsize=(10,5))
    b=plt.bar(alg,data,color=("r","g","b","y","m","black"))
    plt.title("Accuracy comparison of R2L Attacks",fontsize=15)
    plt.legend(b,data,fontsize=9)
    plt.savefig('R2L.png')

In [19]:
graph()

In [20]:

import tkinter
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)
from matplotlib.backend_bases import key_press_handler
from matplotlib.figure import Figure
import numpy as np
root = tkinter.Tk()
root.wm_title("Accuracy plot for R2L Attacks")
fig = Figure(figsize=(10,10),dpi=1)
canvas = FigureCanvasTkAgg(fig, master=root)  
canvas.draw()
canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)
icon=tkinter.PhotoImage(file='R2L.png')   
label=tkinter.Label(root,image=icon)
label.pack()
root.mainloop()